# **Project 2** Dharmesh Shah - MSFT-AI-Professional Program - Dec 24

## **Business Use Case**

### **Problem Statement:**

In the fast-paced environment of healthcare centers, healthcare professionals often face the challenge of quick and accurate diagnosis of patients while managing an ever-increasing volume of medical information. Ensuring that healthcare providers have access to the latest and most comprehensive medical knowledge is crucial for improving patient outcomes and reducing the time needed to make informed decisions.

There are multiple challenges that these professionals encounter daily, a few being

- Information Overload: Medical professionals need to go through vast amounts of data and research to make accurate diagnoses and treatment plans. This can be overwhelming and time-consuming.
- Efficiency: For overall patient care and quality health outcomes, quick and accurate diagnosis is vital, especially in emergency situations.
- Access to Trusted Knowledge: In the ever-evolving healthcare industry, providing access to reliable and up-to-date medical information from renowned manuals and research papers is essential for maintaining high standards of care.


### **Objective:**

A renowned chain of hospitals has decided to leverage AI to build a state-of-the-art solution to help healthcare professionals overcome the aforementioned challenges. They have recruited you as an AI specialist and tasked you with building a RAG-based AI solution that leverages renowned medical manuals as its knowledge base. This AI system will act as a POC towards an end product that’ll assist healthcare professionals in making better, quicker, and more accurate diagnoses, ultimately leading to faster patient resolutions and enabling better patient outcomes by reducing errors in diagnosis, saving valuable time for information retrieval, and standardizing care practices across the board.

### **Questions:**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### **How This Application Empowers Professionals and Elevates Healthcare Organizations**

- **Enhanced Diagnostic Accuracy:**
Provides evidence-based insights for better diagnosis and treatment.

- **Time Efficiency:**
Instantly retrieves critical information, saving valuable time.

- **Improved Patient Care:**
Ensures informed decisions with up-to-date medical knowledge.

- **Cost-Effective Operations:**
Reduces redundant tests and consultation delays, lowering costs.

- **Knowledge Empowerment:**
Keeps doctors updated on the latest advancements.

- **Competitive Edge for the Hospital:**
Positions the hospital as a leader in healthcare innovation.

This collaboration between St. Bernard’s Medical Center and InnoviTech Solutions highlights the transformative potential of AI in revolutionizing healthcare.

## **1. Install and Import Required Libraries**

In [ ]:
# Install the Azure Machine Learning SDK and FAISS-related utilities
%pip install azure-ai-ml
%pip install -U 'azureml-rag[faiss,hugging_face]>=0.2.36'
%pip install azure-identity

## **2. Configure Azure Machine Learning Workspace**

### Get client for AzureML Workspace

In [3]:
#Write you are code here (AzureML and authentication libraries)
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

In [4]:
# Define workspace configuration (replace with your details)
workspace_config = {
    "subscription_id": "d8d51058-2fe2-4f6a-bf7e-c2f2328a6998",
    "resource_group": "GL-Dec24-Pro", 
    "workspace_name": "FitwellWorkspace"
}

In [5]:
%%writefile workspace.json
{
    "subscription_id": "d8d51058-2fe2-4f6a-bf7e-c2f2328a6998",
    "resource_group": "GL-Dec24-Pro", 
    "workspace_name": "FitwellWorkspace"
}

Overwriting workspace.json


In [6]:
# Initialize credentials for Azure authentication
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [7]:
# Initialize the MLClient to connect with AzureML
ml_client = MLClient.from_config(credential=credential, path="workspace.json")

Found the config file in: workspace.json


In [8]:
# Create an AzureML Workspace object
ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)

## **3. Register the Reports Dataset as a Data Asset**

In [9]:
#Write you are code here
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import zipfile
import os

# Path to the ZIP file containing Tesla annual reports
zip_file_path = 'MedicalDiagnosisManuals.zip'

# Directory to extract the reports
extract_to_directory = './extracted_dataset_reports'
os.makedirs(extract_to_directory, exist_ok=True)

# Extract the ZIP file containing the reports
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

# Register the extracted data as a Data asset in AzureML
local_data_path = extract_to_directory
data_asset_name = "Bernards-medical-manuals"
data_asset_description = "A collection of medical manuals used by St. Bernard's Medical Center for embedding generation and knowledge retrieval in the RAG system."

data_asset = Data(
    path=local_data_path,
    type=AssetTypes.URI_FOLDER,  # Registering as a folder URI (AssetTypes.URI_FOLDER)
    description=data_asset_description,
    name=data_asset_name
)

# Use the MLClient to register the data asset
ml_client.data.create_or_update(data_asset)
print(f"Data asset '{data_asset.name}' registered successfully.")

Data asset 'Bernards-medical-manuals' registered successfully.


## **4. Set Up Azure OpenAI Connection**

### Run the cells under _either_ heading (OpenAI or HuggingFace) to use the respective embedding model

#### **Note:**

When selecting an embedding model, ensure consistency between the embedding model used here and the one configured for your vectorstore retriever. Using mismatched models may result in dimension mismatches, leading to errors during vector retrieval. For example:

- If you use the `text-embedding-ada-002` model from Azure OpenAI for embedding creation, ensure the same model is specified for the vectorstore retriever.
- Similarly, if using a HuggingFace model like `all-mpnet-base-v2`, configure the vectorstore retriever with this model.

Maintaining alignment between the embedding model and vectorstore retriever is crucial for the proper functioning of your pipeline.

#### OpenAI

In [10]:
# # Azure Open AI redentials and the id of the deployed chat model are stored as
# # key value pairs in a json file
with open('config.json', 'r') as az_creds: #Fill the blank with json credentails file 
    data = az_creds.read()

# Credentials to authenticate to the personalized Open AI model server
import json
creds = json.loads(data)

In [11]:
from azureml.rag.utils.connections import get_connection_by_name_v2, create_connection_v2

# Define the connection name for Azure OpenAI
aoai_connection_name = "MLS13_December24"

# If the connection doesn't exist, create a new one
target = creds["AZURE_OPENAI_ENDPOINT"]  # Replace with your Azure OpenAI endpoint
key = creds["AZURE_OPENAI_KEY"]          # Replace with your Azure OpenAI API key
api_version = creds["AZURE_OPENAI_APIVERSION"]    # Replace with the appropriate API version

aoai_connection = create_connection_v2(
    workspace=ws,
    name=aoai_connection_name,
    category="AzureOpenAI",
    target=target,
    auth_type="ApiKey",
    credentials={"key": key},
    metadata={"ApiType": "azure", "ApiVersion": api_version},
)

aoai_connection_id = aoai_connection["id"]

print(f"Azure OpenAI connection created or retrieved successfully: {aoai_connection_id}")

Azure OpenAI connection created or retrieved successfully: /subscriptions/d8d51058-2fe2-4f6a-bf7e-c2f2328a6998/resourceGroups/GL-Dec24-Pro/providers/Microsoft.MachineLearningServices/workspaces/FitwellWorkspace/connections/MLS13_December24


To use the Azure OpenAI `text-embedding-ada-002` model for embedding generation, ensure the model has been deployed and is ready for inference. [Follow these instructions](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#deploy-a-model) to deploy the embedding model in your Azure OpenAI resource.

Once deployed, obtain the credentials for the text-embedding-ada-002 model and save them in your `config.json` file. This is necessary for the following code to execute successfully.

Ensure that the `config.json` file includes the appropriate keys:

- `AZURE_OPENAI_EMBEDDING_MODEL`: Name of the Azure OpenAI embedding model.
- `AZURE_OPENAI_EMBEDING_DEPLOYMENT`: Deployment name for the embedding model.

Finally we will combine the deployment and model information into a uri form which the AzureML embeddings components expect as input.

In [12]:
aoai_embedding_model_name = creds["AZURE_OPENAI_EMBEDDING_MODEL"]
aoai_embedding_deployment_name = creds["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
embeddings_model_uri = f"azure_open_ai://deployment/{aoai_embedding_deployment_name}/model/{aoai_embedding_model_name}"
print(f"Embedding Model URI: {embeddings_model_uri}")

Embedding Model URI: azure_open_ai://deployment/text-embedding-ada-002/model/text-embedding-ada-002


#### HuggingFace


In [ ]:
#embeddings_model_uri = "hugging_face://model/sentence-transformers/_________"   #Fill the blank with embedding model

## **5. Setup Pipeline to process data into Index**

### **Define Pipeline Components**

In [13]:
# Write you are code here
# Import the MLClient to access the AzureML registry
ml_registry = MLClient(credential=credential, registry_name="azureml")

# Retrieve components for processing data, generating embeddings, and creating the FAISS index
crack_and_chunk_component = ml_registry.components.get(
    "llm_rag_crack_and_chunk", label="latest"
)
generate_embeddings_component = ml_registry.components.get(
    "llm_rag_generate_embeddings", label="latest"
)
create_faiss_index_component = ml_registry.components.get(
    "llm_rag_create_faiss_index", label="latest"
)
register_mlindex_component = ml_registry.components.get(
    "llm_rag_register_mlindex_asset", label="latest"
)


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [14]:
print(crack_and_chunk_component)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: llm_rag_crack_and_chunk
version: 0.0.78
display_name: LLM - Crack and Chunk Data
description: 'Creates chunks no larger than `chunk_size` from `input_data`, extracted
  document titles are prepended to each chunk


  LLM models have token limits for the prompts passed to them, this is a limiting
  factor at embedding time and even more limiting at prompt completion time as only
  so much context can be passed along with instructions to the LLM and user queries.

  Chunking allows splitting source data of various formats into small but coherent
  snippets of information which can be ''packed'' into LLM prompts when asking for
  answers to user query related to the source documents.


  Supported formats: md, txt, html/htm, pdf, ppt(x), doc(x), xls(x), py

  '
tags:
  Preview: ''
type: command
inputs:
  input_data:
    type: uri_folder
    description: Uri Folder containing files to be chunked.
    op

### **Build the AzureML Pipeline**

In [15]:
# Write you are code here
from azure.ai.ml import Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities._job.pipeline._io import PipelineInput
from typing import Optional

In [16]:
# Utility function for automatic compute configuration
def use_automatic_compute(component, instance_count=1, instance_type="Standard_NC4as_T4_v3"):
    """Configure a component to use automatic compute."""
    component.set_resources(
        instance_count=instance_count,
        instance_type=instance_type,
        properties={"compute_specification": {"automatic": True}},
    )
    return component


# Utility function to check if optional pipeline inputs are provided
def optional_pipeline_input_provided(input: Optional[PipelineInput]):
    """Check if optional pipeline inputs are provided."""
    return input is not None and input._data is not None    

In [20]:
@pipeline(default_compute="serverless")
def diagnosismanuals_to_faiss(
    data_asset_path: str,
    embeddings_model: str,
    asset_name: str,
    chunk_size: int = 1024,
    data_source_glob: str = None,
    document_path_replacement_regex: str = None,
    aoai_connection_id=None,
    embeddings_container=None,
):
    """Pipeline to process Bernards-medical-manuals and create a FAISS vector index."""
    
    # Step 1: Chunk data into smaller pieces
    crack_and_chunk = crack_and_chunk_component(
        input_data=Input(type="uri_folder", path=data_asset_path),  # Input data asset
        input_glob=data_source_glob,
        chunk_size=chunk_size,
        document_path_replacement_regex=document_path_replacement_regex,
    )
    use_automatic_compute(crack_and_chunk)  # Apply compute configuration

    # Step 2: Generate embeddings for the data chunks
    generate_embeddings = generate_embeddings_component(
        chunks_source=crack_and_chunk.outputs.output_chunks,
        embeddings_container=embeddings_container,
        embeddings_model=embeddings_model,
    )
    use_automatic_compute(generate_embeddings)  # Apply compute configuration
    
    # Optional: Include Azure OpenAI connection ID
    if optional_pipeline_input_provided(aoai_connection_id):
        generate_embeddings.environment_variables[
            "AZUREML_WORKSPACE_CONNECTION_ID_AOAI"
        ] = aoai_connection_id
    
    if optional_pipeline_input_provided(embeddings_container):
        generate_embeddings.outputs.embeddings = Output(
            type="uri_folder", path=f"{embeddings_container.path}/{{name}}"
        )

    # Step 3: Create a FAISS vector index from embeddings
    create_faiss_index = create_faiss_index_component(
        embeddings=generate_embeddings.outputs.embeddings,
    )
    use_automatic_compute(create_faiss_index)  # Apply compute configuration

    # Step 4: Register the FAISS index as an MLIndex asset
    register_mlindex = register_mlindex_component(
        storage_uri=create_faiss_index.outputs.index, 
        asset_name=asset_name
    )
    use_automatic_compute(register_mlindex) # Apply compute configuration
    
    return {
        "mlindex_asset_uri": create_faiss_index.outputs.index,
        "mlindex_asset_id": register_mlindex.outputs.asset_id,
    }

## **6.Submit the Pipeline**

In [21]:
# Define the asset name and data source glob pattern
# asset_name = "diagnosis_faiss_index"  #Name for the FAISS index asset created using Huggingface model embeddings.
asset_name = "diagnosis_faiss"  #Name for the FAISS index asset created using OpenAI model embeddings.
data_source_glob = "**/*.pdf"  # Pattern to match input data files

In [22]:
# Get the input data asset path from the workspace datastore
datastore_path = ml_client.data.get(data_asset_name, version="1").path
print(f"Datastore path: {datastore_path}")

Datastore path: azureml://subscriptions/d8d51058-2fe2-4f6a-bf7e-c2f2328a6998/resourcegroups/GL-Dec24-Pro/workspaces/FitwellWorkspace/datastores/workspaceblobstore/paths/LocalUpload/ab0555d33a743acbb9fc52dfbb7b50b1/extracted_dataset_reports/


In [23]:
# Create the pipeline job by calling the defined pipeline function
pipeline_job = diagnosismanuals_to_faiss(
    embeddings_model=embeddings_model_uri,  # URI of the embeddings model
    aoai_connection_id=aoai_connection_id,  # Connection ID for Azure OpenAI (optional)
    embeddings_container=Input(
        type="uri_folder",
        path=f"azureml://datastores/workspaceblobstore/paths/embeddings/{asset_name}"
    ),  # Path for storing generated embeddings
    data_asset_path=Input(
        type="uri_folder",
        path=datastore_path
    ),  # Input data asset path
    chunk_size=1024,  # Size of chunks for processing
    data_source_glob=data_source_glob,  # Glob pattern for input files
    asset_name=asset_name,  # Name of the MLIndex asset
)

In [27]:
# Add properties for better indexing and artifact tracking in the AzureML UI
pipeline_job.properties["azureml.mlIndexAssetName"] = asset_name
pipeline_job.properties["azureml.mlIndexAssetKind"] = "faiss"
pipeline_job.properties["azureml.mlIndexAssetSource"] = "Data asset"

In [28]:
# Submit the pipeline job for execution
submitted_pipeline = ml_client.jobs.create_or_update(pipeline_job)
print(f"Pipeline submitted successfully! Job ID: {submitted_pipeline.id}")

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
pathOnCompute is not a known attribute

Pipeline submitted successfully! Job ID: /subscriptions/d8d51058-2fe2-4f6a-bf7e-c2f2328a6998/resourceGroups/GL-Dec24-Pro/providers/Microsoft.MachineLearningServices/workspaces/FitwellWorkspace/jobs/olive_muscle_fj1n540srv


In [29]:
# Stream the pipeline job logs for real-time monitoring
ml_client.jobs.stream(submitted_pipeline.name)

RunId: olive_muscle_fj1n540srv
Web View: https://ml.azure.com/runs/olive_muscle_fj1n540srv?wsid=/subscriptions/d8d51058-2fe2-4f6a-bf7e-c2f2328a6998/resourcegroups/GL-Dec24-Pro/workspaces/FitwellWorkspace

Streaming logs/azureml/executionlogs.txt

[2025-04-21 01:18:26Z] Completing processing run id cb672b76-bebb-4d05-9ff5-5cd9ce06541e.
[2025-04-21 01:18:27Z] Completing processing run id fdd51e1a-66d0-45d3-a6b8-8e08f74c0054.
[2025-04-21 01:18:28Z] Completing processing run id ee727436-33d2-4cf9-a0ad-ca3a48cf1f85.
[2025-04-21 01:18:29Z] Completing processing run id 53dc3b48-acf4-46d9-be0f-154298413d23.
[2025-04-21 01:18:30Z] Finishing experiment: no runs left and nothing to schedule.

Execution Summary
RunId: olive_muscle_fj1n540srv
Web View: https://ml.azure.com/runs/olive_muscle_fj1n540srv?wsid=/subscriptions/d8d51058-2fe2-4f6a-bf7e-c2f2328a6998/resourcegroups/GL-Dec24-Pro/workspaces/FitwellWorkspace



## **7.Execution Pipeline**

[Jobs](https://huggingface.co/spaces/fordharmesh/Intelligent_reporting_on_azure/resolve/main/RAGProjectExecutionPiplelineList.jpg)

![Jobs](https://huggingface.co/spaces/fordharmesh/Intelligent_reporting_on_azure/resolve/main/RAGProjectExecutionPiplelineList.jpg)


[Pipeline](https://huggingface.co/spaces/fordharmesh/Intelligent_reporting_on_azure/resolve/main/RAGProjectExecutionPipleline.jpg)

![Pipeline](https://huggingface.co/spaces/fordharmesh/Intelligent_reporting_on_azure/resolve/main/RAGProjectExecutionPipleline.jpg)

# **Information Retrieval and Response Generation Using LangChain-FAISS and Azure OpenAI**

## **1.Installing Required Libraries**

In [30]:
# Install the required LangChain and HuggingFace libraries
%pip install -U langchain-community
%pip install -U langchain-huggingface
%pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## **2. Setting Up Data Retrieval**

### **Downloading and Setting Up FAISS Index Assets**


In [31]:
# Import necessary utilities for artifact retrieval
import azure.ai.ml._artifacts._artifact_utilities as artifact_utils

# Retrieve the path to the latest FAISS index asset from Azure ML
data_info = ml_client.data.get(name=asset_name, label="latest").path

# Download the FAISS index asset to a local directory
artifact_utils.download_artifact_from_aml_uri(
    uri=data_info,
    destination="./diagnosisfaissindexasset/",
    datastore_operation=ml_client.datastores
)


'./diagnosisfaissindexasset/'

## **3. Loading the FAISS Index**

### **Loading the FAISS Index and Preparing the Retriever**

We load the FAISS index from the downloaded files and connect it to an embedding model. This embedding model ensures that queries are converted into vector space to match the stored documents effectively.

### **Note:**
When loading the FAISS index and setting up the retriever, it is crucial to use the same embedding model that was used during the creation of the FAISS index. This ensures that the dimensionality of the embeddings produced by the query matches the dimensionality of the embeddings stored in the FAISS index.

If a different embedding model is used between these two steps, a dimension mismatch will occur, leading to errors in retrieving relevant documents or performing similarity searches.

In [32]:
# Path to the directory containing FAISS index files
index_folder_path = "./diagnosisfaissindexasset/"

In [ ]:
#from langchain_huggingface import HuggingFaceEmbeddings
#
## Specify the embedding model used during FAISS index creation
#embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
#embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [34]:
# from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings

# # Specify the embedding model used during FAISS index creation
embedding_model = AzureOpenAIEmbeddings(
    model=creds["AZURE_OPENAI_EMBEDDING_MODEL"],
    azure_endpoint= creds["AZURE_OPENAI_ENDPOINT"],
    api_key= creds["AZURE_OPENAI_KEY"],
    openai_api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [35]:
from langchain.vectorstores import FAISS

# Load the FAISS index and associate it with the embedding model
retriever = FAISS.load_local(
    folder_path=index_folder_path, 
    embeddings=embedding_model, 
    allow_dangerous_deserialization=True  # Acknowledge the source of the data for safe loading
)

# The retriever is now ready to perform similarity searches.

## **4. Performing a Similarity Search**

In [36]:
# Define a query to test the retriever
query = "How to diagnos Nutritional Disorders"

# Retrieve the top 3 most relevant documents
results = retriever.similarity_search(query, k=3)

# Display the results
for doc in results:
    print(f"Document: {doc.page_content}\nMetadata: {doc.metadata}")


Document: Title: The_Merck_Manual_of_Diagnosis_and_Therapy_2011 - 19th Edn........pdfDiagnosis is based on results of medical and diet histories, physical examination, body composition
analysis (see p. 
58
), and selected laboratory tests.
History:
 History should include questions about dietary intake (see
Fig. 2-1
), recent changes in weight, and risk factors for undernutrition, including drug and alcohol use.
Unintentional loss of ≥ 10% of usual body weight during a 3-mo period indicates a high probability of
undernutrition. Social history should include questions about whether money is available for food and
whether the patient can shop and cook.
Review of systems should focus on symptoms of nutritional deficiencies (see 
Table 2-1
). For example,
impaired night vision may indicate vitamin A deficiency.
Physical examination:
 Physical examination should include measurement of height and weight,
inspection of body fat distribution, and anthropometric measurements of lean body mass. 

## **5: Creating the System and User Prompt Templates**

In [37]:
# Write you are code here
# Define the system prompt for the Azure OpenAI model
qna_system_message = """
You are an assistant to a doctor. Your task is to summarize and provide relevant information to doctor's question based on the provided context.

User input will include the necessary context for you to answer their questions. This context will begin with the token: ###Context.
The context contains references to specific portions of documents relevant to the user's query, along with page number from the report.
The source for the context will begin with the token ###Page

When crafting your response:
1. Select only context relevant to answer the question.
2. Include the source links in your response.
3. User questions will begin with the token: ###Question.
4. If the question is irrelevant or if the context is empty - "Sorry, this is out of my knowledge base"

Please adhere to the following guidelines:
- Your response should only be about the question asked and nothing else.
- Answer only using the context provided.
- Do not mention anything about the context in your final answer.
- If the answer is not found in the context, it is very very important for you to respond with "Sorry, this is out of my knowledge base"
- If NO CONTEXT is provided, it is very important for you to respond with "Sorry, this is out of my knowledge base"

Here is an example of how to structure your response:

Answer:
[Answer]

Page:
[Page number]
"""

# Define the user message template
qna_user_message_template = """
###Context
Here are some documents and their page number that are relevant to the question mentioned below.
{context}  

###Question  
{question}  
"""

## **6. Generating the Response**

In [38]:
# Install the rquired packages
%pip install openai==1.55.3 tiktoken==0.6 session-info --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-rag 0.2.38 requires tiktoken<1.0,>=0.7, but you have tiktoken 0.6.0 which is incompatible.
langchain-openai 0.3.14 requires openai<2.0.0,>=1.68.2, but you have openai 1.55.3 which is incompatible.
langchain-openai 0.3.14 requires tiktoken<1,>=0.7, but you have tiktoken 0.6.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [39]:
# Import required libraries
import json
import tiktoken
import pandas as pd
from openai import AzureOpenAI

In [40]:
# Load Azure OpenAI credentials
with open('config-copy.json', 'r') as az_creds:
    data = az_creds.read()

creds = json.loads(data)

In [41]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [42]:
 def generate_rag_response(user_input):
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.similarity_search(user_input, k=3)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    # Compose the prompt
    prompt = [
        {'role': 'system', 'content': qna_system_message},
        {'role': 'user', 'content': qna_user_message_template.format(
            context=context_for_query,
            question=user_input
            )
        }
    ]

    # Generate the response using Azure OpenAI
    try:
        response = client.chat.completions.create(
            model=creds["CHATGPT_MODEL"],
            messages=prompt,
            temperature=0
        )

        # Extract and print the model's response
        response = response.choices[0].message.content.strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'


    print(response)

#### Question 1: What is the protocol for managing sepsis in a critical care unit?

In [43]:
user_input = "What care and protocol should be follow to manage sepsis in a critical care unit?"    # Enter the question to be answered by the system here 
generate_rag_response(user_input)

Answer:
In a critical care unit, the management of sepsis includes the following protocols:

1. **Prompt Empiric Therapy**: Initiate parenteral antibiotics immediately after obtaining specimens for Gram stain and culture. The choice of antibiotics should be based on the suspected source, clinical setting, and previous culture results.

2. **Antibiotic Regimens**: For septic shock of unknown cause, consider using gentamicin or tobramycin with a 3rd-generation cephalosporin. If resistant staphylococci or enterococci are suspected, add vancomycin. Include drugs effective against anaerobes if there is an abdominal source.

3. **Surgical Intervention**: Drain abscesses and surgically excise necrotic tissues to eliminate septic foci, as antibiotic therapy alone may not be sufficient.

4. **Glucose Management**: Normalize blood glucose levels using a continuous IV insulin infusion to maintain glucose between 80 to 110 mg/dL.

5. **Corticosteroid Therapy**: Administer replacement doses of cort

#### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [44]:
user_input = "Please list common symptoms for appendicitis. Can we cure this via medicine? If not, then please describe as what surgical procedure should we follow to treat it."   # Enter the question to be answered by the system here 
generate_rag_response(user_input)

Common symptoms of appendicitis include:
- Epigastric or periumbilical pain followed by nausea, vomiting, and anorexia.
- Pain that shifts to the right lower quadrant after a few hours.
- Pain increases with cough and motion.
- Right lower quadrant direct and rebound tenderness at McBurney's point.
- Additional signs such as Rovsing sign, psoas sign, and obturator sign.
- Low-grade fever (rectal temperature 37.7 to 38.3° C [100 to 101° F]).

Appendicitis cannot be cured via medicine alone. The treatment for acute appendicitis is surgical removal, which can be performed either through open or laparoscopic appendectomy. It is crucial to perform the surgery promptly, as treatment delay increases mortality.

Page:
164-165


#### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [45]:
user_input = "Please list out effective treatments for addressing sudden patchy hair loss, commonly seen as localized blad spots. What could be possible causes behind it?"   # Enter the question to be answered by the system here 
generate_rag_response(user_input)

Answer:
Effective treatments for sudden patchy hair loss (alopecia areata) include:
- Corticosteroids (intralesional or topical)
- Topical anthralin
- Minoxidil
- Induction of allergic contact dermatitis using diphencyprone or squaric acid dibutylester

Possible causes behind sudden patchy hair loss include:
- Autoimmune disorders
- Environmental triggers (such as infection or emotional stress)
- Coexisting autoimmune conditions (e.g., vitiligo or thyroiditis)

Page:
849


#### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [46]:
user_input = "Please list out recommended treatment for physicqal injury to patients brain tissue that resulted in temporary or permanent impairment of brain function."   # Enter the question to be answered by the system here 
generate_rag_response(user_input)

Answer:
The recommended treatment for traumatic brain injury (TBI) includes:

1. Ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure.
2. Surgery may be needed for severe injuries to:
   - Place monitors to track and treat intracranial pressure.
   - Decompress the brain if intracranial pressure is increased.
   - Remove intracranial hematomas.
3. In the first few days post-injury, focus on maintaining adequate brain perfusion and oxygenation, and preventing complications of altered sensorium.
4. Rehabilitation may be required for many patients after the initial treatment.

Page:
3395-3400


#### Question 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [47]:
user_input = "Please list out necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"   # Enter the question to be answered by the system here 
generate_rag_response(user_input)

Answer:
For a person who has fractured their leg during a hiking trip, the following precautions and treatment steps should be taken:

1. **Immediate Care**:
   - **Rest**: Prevent further injury by avoiding movement.
   - **Ice**: Apply ice to minimize swelling and pain (15 to 20 minutes every 1-2 hours).
   - **Compression**: Use an elastic bandage or splint to compress the injury.
   - **Elevation**: Keep the injured leg elevated above heart level to reduce swelling.

2. **Immobilization**:
   - Use a splint to immobilize the leg and prevent further injury.
   - A cast may be required for long-term immobilization depending on the severity of the fracture.

3. **Medical Evaluation**:
   - Seek medical attention for a thorough evaluation and treatment.
   - X-rays may be necessary to assess the fracture.
   - Treatment may involve reduction (realignment of the bone) and possibly surgical intervention if the fracture is severe.

4. **Pain Management**:
   - Pain can be managed with med

#### Power Ahead!